# Conditional Model

CS4200 Final Project 

https://www.cpp.edu/~dmhughes/cs4200_project/blogpost.html

**Connecting to Google Drive**

In [0]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 20.9MB/s 
  Running setup.py bdist_wheel for PyDrive ... - done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
if os.path.exists("winemag-data-130k-v2.csv"):
  print("Wine csv already exists")
else:
  download_test = drive.CreateFile({'id': '127NdtnUoFh_nAJOwYaj_5i9uBof2lwe1'})
  wine_file = download_test.GetContentFile('wine.zip')
  #print('Downloaded content "{}"'.format(len(wine_file)))

  !unzip wine.zip

Archive:  wine.zip
  inflating: winemag-data-130k-v2.csv  


In [0]:
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation,CuDNNLSTM,Dropout, GRU, CuDNNGRU, Flatten
from keras.callbacks import LambdaCallback, ModelCheckpoint, ReduceLROnPlateau,Callback, History
from keras.optimizers import Adam
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
import numpy as np
import pandas as pd
import random
import sys
import io
import os
import matplotlib.pyplot as plt
import pickle
import math
import time

Using TensorFlow backend.


**Condtioning context functions (for price)**

In [0]:
def create_price_category(min_price, max_price, num_cat):
  cat_size = int((max_price - min_price) / num_cat)
  temp = [min_price]
  for i in range(1, num_cat):
    temp.append(min_price + i*cat_size)
  return temp

# print("max ",price_df.max(), " min ", price_df.min() )    
# price_cats = create_price_category(price_df.min(), price_df.max(), 20)
# print(price_cats, len(price_cats))

def price_to_category(price, price_cats):
  price_i = 0
  
  if price < price_cats[0]:
    price_i = 0
  elif price > price_cats[len(price_cats)-1]:
    price_i = len(price_cats)-1
  else:
    for i in range(1, len(price_cats)):
      if price >= price_cats[i-1] and price <= price_cats[i]:
        if abs(price - price_cats[i-1]) > abs(price - price_cats[i]):
          price_i = i
          break
        else:
          price_i = i-1
          break
    
        
  return price_i
  
# print(125, " ", cats[priceCategory(125, cats)] ) 
# print(109, " ", cats[priceCategory(109, cats)] ) 
# print(700, " ", cats[priceCategory(700, cats)] ) 
# print(14, " ", cats[priceCategory(14, cats)] ) 

**Handle the CSV as a pandas dataframe**

**Also create a list of [(character, price)]**

**Create vocab data structures**

In [0]:
df = pd.read_csv('winemag-data-130k-v2.csv')
print(list(df)) # column names 
# get wine reviews that have points>=94 
df= df[df['points']>=94]
df= df[df['country']=="US"]
print (df.shape)

num_sample=df.shape[0] 

price_df = df[df.columns[5]]
print("max ",price_df.max(), " min ", price_df.min() )


#get the description column that contains win reviews
desc_df = df[df.columns[2]]
print("First description: ", desc_df.values[0])

# remove new line characters
desc_df= desc_df.replace({r'\n+': ''}, regex=True)

# remove duplicated reviews
desc_df=desc_df.drop_duplicates()
print("After dropping duplicates: ", desc_df.shape)

text_file = open("wine_review_text.txt", "w")
for line in desc_df.values:
  text_file.write(line)
text_file.close()


text_file = open("wine_review_text.txt", "r")
text = text_file.read()
length_text = len(text)
print("length_text (the number of characters)", length_text)
#get the set of characters in the language
set_chars = sorted(list(set(text)))
print("NumChars in lang = ",len(set_chars),", set of chars in the lang: ", set_chars)
char_to_int = dict((c, i) for i, c in enumerate(set_chars))
int_to_char = dict((i, c) for i, c in enumerate(set_chars))

cut_length=60
num_price_cats = 20

price_cats = create_price_category(price_df.min(), price_df.max(), num_price_cats)

combined_text_price = []  
for i, line in enumerate(desc_df.values):
  for ch in line:
    combined_text_price.append((char_to_int[ch], price_to_category(price_df.values[i], price_cats)))

print(combined_text_price[1000:1500])    
    
#print("combined_text_price", combined_text_price[0])
del desc_df
del df
  

['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title', 'variety', 'winery']
(2787, 14)
max  625.0  min  16.0
First description:  Citrus-kissed saltiness lies at the core of this incredibly delicious, deliriously crafted wine, from the great grape grower in the heart of the Russian River Valley. Beautifully crisp, it lingers in stony minerality and a freshness of apricot, memorable from first sip to last.
After dropping duplicates:  (2715,)
length_text (the number of characters) 932982
NumChars in lang =  101 , set of chars in the lang:  [' ', '!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', '

**Sequence splitting function**

In [0]:
#break it down  the text into chunks of 90k characters each
def get_data_from_chunk_cond(combined_text_price, step=3, num_one_hots=2):
  
  seqs = []
  next_chars_to_seq = []

  for i in range(0, len(combined_text_price)-cut_length, step):
    seqs.append(combined_text_price[i: i+cut_length])
    next_chars_to_seq.append(combined_text_price[i+cut_length])

  print("Number of sequences: ", len(seqs) )  

  #create vectors X and y for fitting model
  X = np.zeros((len(seqs), cut_length, num_one_hots), dtype=np.uint)
  y = np.zeros((len(seqs), num_one_hots), dtype=np.uint)
  for i, seq in enumerate(seqs):
      for j, pair in enumerate(seq):
        X[i, j, 0] = pair[0]
        X[i, j, 1] = pair[1]
      y[i, 0] = next_chars_to_seq[i][0]
      y[i, 1] = next_chars_to_seq[i][1]

      
  X_char = np.array([ [word[0]  for word in seq]  for seq in X])
  X_price = np.array([ [seq[-1][1]] for seq in X])
  y_ = np.array([keras.utils.to_categorical(word[0], len(set_chars), dtype='int32')  for word in y])
  
  del seqs
  del next_chars_to_seq
  del X
  del y
  
  return [X_char, X_price, y_]




** Build Condtional model in functional Keras style**

In [0]:
from keras import backend as K
K.clear_session()

print('Building Model')
#first input layer
char_input = Input(shape=(None,), dtype='int32', name='char_input')
char_embedd = Embedding(output_dim=10, input_dim=cut_length)(char_input)

x = CuDNNLSTM(256, return_sequences=True)(char_embedd)
x = Dropout(0.3)(x)
char_encoded = CuDNNLSTM(256)(x)

#second input layer
price_input = Input(shape=(1,), name='price_input')

#combine the output of the left and right models
combined = keras.layers.concatenate([char_encoded, price_input])

x = Dense(128, activation='relu')(combined)
x = Dropout(0.3)(x)
final_output = Dense(len(set_chars), activation='softmax')(x)

print(final_output.shape)

model = Model(inputs=[char_input, price_input], outputs=final_output)

def load_model():
  
  if os.path.exists("weights.hdf5"):
    print("Loading transfer weights...")
    model.load_weights("weights.hdf5")
  else:
    print("Starting new model weights...")

  model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Building Model
(?, 101)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 10)     600         char_input[0][0]                 
__________________________________________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)        (None, None, 256)    274432      embedding_1[0][0]                
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, None, 256)    0           cu_dnnlstm_1[0][0]               
_____________________________________________________________________________________

**Addtional Keras Functions**
- sample() provides for choosing a character based on temperature/diversity
- on_train_end() generates text at the end of training the model
- also various Keras callbacks

In [0]:
#Function to sample index from probab array
#    -temp is a parameter used in softmax func which controls level of newness
#    -temp=1 constricts sampling and leads to less diverse/more repetitive text
#i.e.
#The “sampling index” process will add some variety to the final result by generating some randomness with the given prediction.
#If the temperature is very small, it will always pick the index with highest predicted probability.
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64') #convert one-hot encoding to an array of float64
    preds = np.log(preds) / temperature         #log each value elementwise then perform elementwise division by the temperature
    #apply softmax
    exp_preds = np.exp(preds)                   
    preds = exp_preds / np.sum(exp_preds)
    #create a one-hot encoding
    probas = np.random.multinomial(1, preds, 1) 
    return np.argmax(probas)      #argmax = Returns the indices of the maximum values along an axis.                


# Function called at the end of an training
def on_train_end(_):
    #print('\n----- Generating text after Epoch: %d' % epoch)
    start_i = random.randint(0, length_text - cut_length - 1)
    
    sentence = combined_text_price[start_i: start_i + cut_length]
    price_i = sentence[-1][1]
    sentence_in_chars =''.join ([int_to_char[c[0]] for c in sentence])
    print('----- Generating with seed: "' , sentence_in_chars , '" and price=' , price_cats[price_i] , ':')
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)
        generated = ''
        sentence = combined_text_price[start_i: start_i + cut_length]
        price_i = sentence[-1][1]
        sentence_in_chars =''.join ([int_to_char[c[0]] for c in sentence])
        generated += sentence_in_chars
        sys.stdout.write(generated)
        for i in range(cut_length):
            X_pred_char = np.array([letter[0] for letter in sentence])
            X_pred_char = np.expand_dims(X_pred_char, axis=0)
            X_pred_price = np.array([price_i])                      #maybe price_i will work here
            X_pred_price = np.expand_dims(X_pred_price, axis=0)
            preds = model.predict([X_pred_char,X_pred_price], verbose=0)[0]
            next_index = sample(preds, diversity)
            #print(" index:",next_index )
            next_char = int_to_char[next_index]

            generated += next_char
            sentence = sentence[1:] + [(next_index,price_i)]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

        
#callback list        
history = History()

#lambdacallback = LambdaCallback()
lambdacallback = LambdaCallback(on_train_end=on_train_end)

filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, min_lr=0.00001, verbose=1)

callbacks = [history, checkpoint, reduce_lr, lambdacallback]


**Functions to load loss history from file in order to continue training from a weights file**

In [0]:
def load_loss_history(file_loss= "lossHistory"):
  #open the loss history report file for reading
  history_dict={}

  if os.path.exists(file_loss):
    file_object= open(file_loss,'rb') 
    history_dict = pickle.load(file_object)
    file_object.close()
    print("Loading loss history report...")

  else:
    print("Starting new loss history report...")
  
  return history_dict



In [0]:
def save_loss_history(history_dict,file_loss= "lossHistory"):
  
  if(history_dict):
    history_dict['loss'].extend(history_model.history['loss'])
    #history_dict['val_loss'].extend(history_model.history['val_loss'])
  else:
    history_dict=history_model.history
  
  # open the loss history report file for writing
  fileObject = open(file_loss,'wb') 

  # write the object a to the lossHistory file
  pickle.dump(history_dict,fileObject)
  fileObject.close()
  print("Saving loss history report ...")
  return history_dict


In [0]:
print("Chunk Data into X and y")
X_char, X_price, y = get_data_from_chunk_cond(combined_text_price)  

print(X_char.shape, len(X_char), X_char[0])
print(X_price.shape, len(X_price), X_price[0])
print(y.shape, len(y), y[0])

Chunk Data into X and y
Number of sequences:  310974
(310974, 60) 310974 [29 61 72 70 73 71 11 63 61 71 71 57 56  0 71 53 64 72 61 66 57 71 71  0
 64 61 57 71  0 53 72  0 72 60 57  0 55 67 70 57  0 67 58  0 72 60 61 71
  0 61 66 55 70 57 56 61 54 64 77  0]
(310974, 1) 310974 [2]
(310974, 101) 310974 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


**Call the compile, fitting, and saving function**

In [0]:
!rm weights.hdf5
!rm lossHistory

batch_size=128
load_model()
history_dict=load_loss_history()

print("fit model")
start_time = time.time()
history_model=model.fit([X_char,X_price], y, batch_size=128, epochs=50, callbacks=callbacks, verbose=1, validation_split=0) 
end_time = time.time()
print("Wall Time to fit model: ", end_time - start_time)


history_dict=save_loss_history(history_dict)


rm: cannot remove 'weights.hdf5': No such file or directory
rm: cannot remove 'lossHistory': No such file or directory
Starting new model weights...
Starting new loss history report...
fit model
Epoch 1/50
310974/310974 [==============================] - 136s 437us/step - loss: 2.7729

Epoch 00001: loss improved from inf to 2.77293, saving model to weights.hdf5
Epoch 2/50
310974/310974 [==============================] - 131s 423us/step - loss: 2.3539

Epoch 00002: loss improved from 2.77293 to 2.35391, saving model to weights.hdf5
Epoch 3/50
310974/310974 [==============================] - 131s 422us/step - loss: 2.1942

Epoch 00003: loss improved from 2.35391 to 2.19415, saving model to weights.hdf5
Epoch 4/50
310974/310974 [==============================] - 134s 432us/step - loss: 2.1060

Epoch 00004: loss improved from 2.19415 to 2.10599, saving model to weights.hdf5
Epoch 5/50
310974/310974 [==============================] - 229s 738us/step - loss: 2.0417

Epoch 00005: loss improve

In [0]:
model.load_weights("weights.hdf5") #load saved model

**Function that does a similar function to on_train_end but can be called as long as the model has been loaded**

In [0]:
def make_pred(list_of_prices):
  
  start_i = random.randint(0, len(text) - cut_length - 1)
  
  for passed_price in list_of_prices:
    price_i = price_to_category(passed_price, price_cats)
    generated = ''
    sentence = combined_text_price[start_i: start_i + cut_length]
    sentence_in_chars =''.join ([int_to_char[c[0]] for c in sentence])
    generated += sentence_in_chars
    print('\n----- Generating with seed: "' , sentence_in_chars , '" and price=' , price_cats[price_i] , ':\n')

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)
        generated = ''
        sentence = combined_text_price[start_i: start_i + cut_length]
        sentence_in_chars =''.join ([int_to_char[c[0]] for c in sentence])
        generated += sentence_in_chars
        sys.stdout.write(generated)
        for i in range(400):
            X_pred_char = np.array([letter[0] for letter in sentence])
            X_pred_char = np.expand_dims(X_pred_char, axis=0)
            #print("\nX_pred_char",  X_pred_char.shape)
            X_pred_price = np.array([price_i])                      #maybe price_i will work here
            X_pred_price = np.expand_dims(X_pred_price, axis=0)
            #print("X_pred_price", X_pred_price.shape, X_pred_price)
            preds = model.predict([X_pred_char,X_pred_price], verbose=0)[0]
            next_index = sample(preds, diversity)
            #print(" index:",next_index )
            next_char = int_to_char[next_index]

            generated += next_char
            sentence = sentence[1:] + [(next_index,price_i)]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
      
make_pred([600,10])




----- Generating with seed: " ing, a standout for the producer and the appellation in a pi " and price= 586.0 :

----- diversity: 0.2
ing, a standout for the producer and the appellation in a pine of tanoiy fla

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


vors of black cherryes, cherries, cassis, coerries, cherries, cassis, ricoa, coovamon and chocolate and coerry flavors ard a sine tf coanberry and cherry fravors are teartess and tiasted in tas ard soe finish. The tine is a roue tiat wase ih the firit and a sine of caramel and chonamon flavors are tearty and sine coerry flavors ard a sine of chanberry and cherry fravors ard a sont 
----- diversity: 0.5
ing, a standout for the producer and the appellation in a pise if sote ran and black currant flavors. Al ihsm a barance oh ohe finish. It sool the fiui bodied, and the trne is tanered and complex flavors of cherryes, cherries and cassis fluit and teat and cherry fravors are weasyng a seartess tight oht fin al heast tie yise if this iine ii iight and seas oaner of tav, and she salnins are sire and al inght iite if the alpmas flavors of ttoe pite and pan fla
----- diversity: 1.0
ing, a standout for the producer and the appellation in a pitue tf seach and roange seel ti oligerine, exegant f

In [0]:
# print (history_dict['loss'])
# # Plot training & validation loss values
# plt.plot(history_dict['loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train'], loc='upper left')
# plt.show()

**Print the Perplexity over epochs of the currently trained model**

In [0]:
perp = [ math.exp(loss) for loss in history_dict['loss']]
val_perp = [ math.exp(loss) for loss in history_dict['val_loss']]

print ("perplexity:", perp)
print ("validation perplexity:", val_perp)
plt.plot(perp)
plt.plot(val_perp)
plt.title('Model Perplexity')
plt.ylabel('Perplexity')
plt.xlabel('Epoch')
plt.legend(['Train perplexity','Validation perplexity'], loc='upper right')
#plt.legend(['Train perplexity'], loc='upper right')
plt.rcParams["figure.figsize"] = (15,10)
plt.show()

# SAVED GRAPH SECTION

In [0]:
# perp = 
# print ("perplexity:", perp)

# plt.plot(perp)
# plt.title('Condtional Model Perplexity')
# plt.ylabel('Perplexity')
# plt.xlabel('Epoch')
# plt.legend(['Train perplexity'], loc='upper right')
# plt.rcParams["figure.figsize"] = (15,10)
# plt.show()

In [0]:
perp = [16.348418804440428, 10.283687330824307, 8.709842266083166, 7.916933568645355, 7.421489441782192, 7.0404791701003795, 6.739283222542519, 6.494184203938808, 6.2912496292069084, 6.130500311274162, 5.979295128491181, 5.847670846711126, 5.723765428966378, 5.62163165469696, 5.564947735036407, 5.4318938885554395, 5.364175970852322, 5.278832602977237, 5.2275725617891355, 5.159435395295851, 5.099691699359824, 5.035057344278189, 4.984605179391719, 4.940551927997494, 4.898210505625583, 4.856868933861249, 4.811558244906741, 4.770864561923012, 4.742223147979551, 4.705998821525545, 4.682146592487723, 4.6624839233195665, 4.616949919113042, 4.584394430807733, 4.564513030359467, 4.535377087525101, 4.5187681067763386, 4.500901572786341, 4.468717659964723, 4.447586923984543, 4.4373699989993876, 4.415552215277983, 4.397534535170213, 4.384635703645977, 4.372508947468803, 4.34837393164133, 4.34370858916281, 4.321766889744905, 4.312977204539631, 4.297035166427873]
val_perp =[11.221486041894705, 8.640195844811451, 7.65738402690025, 7.155362349899574, 6.741637624743697, 6.5032339376385675, 6.312858631201043, 6.173395964670178, 6.021321080286697, 5.948594937527857, 5.8527591407302175, 5.790766084590234, 5.755720707315652, 5.784654767761994, 5.67536758977052, 5.662697454412181, 5.662634228800737, 5.654726348535229, 5.595754074359111, 5.635172115609468, 5.617537464400125, 5.616668595403047, 5.576405135485581, 5.573847885684787, 5.670847266572062, 5.603106438476108, 5.644738358120938, 5.5908614390470195, 5.6311269157846295, 5.609611333536659, 5.687124503827853, 5.647082764722481, 5.674957354060339, 5.663237839885458, 5.689583701120254, 5.736930756141362, 5.6923525847762315, 5.731692069938086, 5.739202543210095, 5.732914151733973, 5.7313379869139975, 5.756671105145688, 5.743319866446593, 5.792162975412888, 5.783097921228845, 5.799358457756594, 5.785352625979515, 5.778316651320201, 5.809363764388644, 5.874493685157506]

#print ("perplexity:", perp)
#print ("validation perplexity:", val_perp)
plt.plot(perp)
plt.plot(val_perp)
plt.title('Condtional Model Perplexity')
plt.ylabel('Perplexity')
plt.xlabel('Epoch')
plt.legend(['Train perplexity','Validation perplexity'], loc='upper right')
plt.rcParams["figure.figsize"] = (10,8)
plt.show()

**Code to print model graph**
- note that the libraries must be installed and the runtime restarted in order for the pydot error to not be thown

In [0]:
# !pip install -q pydot
# !pip install graphviz 
# !apt-get install graphviz

# from keras.utils import plot_model
# plot_model(model, to_file='model.png')